# Capstone Project Notebook

Jupyter Notebook created for the Applied Data Science Capstone Project with Coursera.

In [4]:
import pandas as pd
import numpy as np

print("Hello Capstone Project Course!")

Hello Capstone Project Course!


# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto. For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. You will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset.

Once the data is in a structured format, you can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.

# PART 1
### CREATE THE DATAFRAME

In [5]:
import requests
from bs4 import BeautifulSoup

URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
res = requests.get(URL).text
soup = BeautifulSoup(res,'lxml')


I save the Wikipedia table into My_table. 
You can locate the table from the label "<table" and the class "wikitable sortable".

In [8]:
My_table = soup.find('table',{'class':'wikitable sortable'})
#My_table

Now, for each row in the table, I save in 3 different lists (p_code, Borough, Neighborhood) the data

In [9]:
p_code=[]
Borough=[]
Neighborhood=[]

for row in My_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        p_code.append(cells[0].find(text=True).rstrip())
        Borough.append(cells[1].find(text=True).rstrip())
        Neighborhood.append(cells[2].find(text=True).rstrip())

Using pandas, I can create a dataframe with 3 columns: PostalCode, Borough, Neighborhood

In [10]:
df=pd.DataFrame(p_code,columns=['PostalCode'])
df["Borough" ]=Borough
df["Neighborhood"]=Neighborhood
df.head(20)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [11]:
comparison_column_N = np.where(df["Neighborhood"] == "Not assigned", True, False)
comparison_column_B = np.where(df["Borough"] == "Not assigned", True, False)
comparison_column_B == comparison_column_N


array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In our case, all the cell that has a Not assigned neighborhood they have also a Not Assigned Borough and viceversa.


In [12]:
df["PostalCode"].unique

<bound method Series.unique of 0      M1A
1      M2A
2      M3A
3      M4A
4      M5A
      ... 
175    M5Z
176    M6Z
177    M7Z
178    M8Z
179    M9Z
Name: PostalCode, Length: 180, dtype: object>

In [13]:
df.shape

(180, 3)

The DataFrame has 3 columns and 180 rows